<a href="https://colab.research.google.com/github/samyam200200/python/blob/master/CrossValidateANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#import
dataset=pd.read_csv("Churn_Modelling.csv")
X=dataset.iloc[:,3:13].values
Y=dataset.iloc[:,13].values

In [3]:
#one hot encoding variables
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
# Country column
ct = ColumnTransformer([("Geo", OneHotEncoder(), [1])], remainder = 'passthrough')
X = ct.fit_transform(X)

# Male/Female
labelencoder_X = LabelEncoder()
X[:, 4] = labelencoder_X.fit_transform(X[:, 4])
X = X.astype('float64')

In [4]:
#removing 1 dummy varibale ie 0 
X=X[:,1:]

In [5]:
#Splitting data
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test=train_test_split(X, Y, test_size=0.20, random_state=0)

In [6]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [7]:
#Making ANN
# Importing Keras
import keras

Using TensorFlow backend.


In [8]:
from keras.models import Sequential
from keras.layers import Dense

In [9]:
#Initalizing ANN
classifier=Sequential()

In [10]:
# Adding first layer -  input layera and first hidden layer
# units = output_dim (i.e. number of hidden layers equal to inputlayers(11)+outputlayers(1)/2)
# init=kernel_initializer (initial weights)
# input_dim (input layers)
classifier.add(Dense(units=6, kernel_initializer="uniform", activation="relu", input_dim=11))

In [11]:
#adding new layer
#this is added to previous neural network, its not needed to add input layer
classifier.add(Dense(units=6, kernel_initializer="uniform", activation="relu"))

In [12]:
#output layer
#softmax for more than one output
classifier.add(Dense(units=1, kernel_initializer="uniform", activation="sigmoid"))

In [13]:
#Compiling ANN
#adam is for stochastic gradient descent
#loss function is for the weights, here for logistic regression we will use lograthmic loss
#more than 2 varibale categorical_crossentropy
#what is used to improve the goodness of model
classifier.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [14]:
# fitting ANN into the dataset
#also pass batch size
# and pass epoch (repeating previous spteps this many times)
classifier.fit(X_train, Y_train, batch_size=10, nb_epoch=100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Epoch 1/100
8000/8000 [==============================] - 4s 467us/step - loss: 0.4828 - accuracy: 0.7959
Epoch 2/100
8000/8000 [==============================] - 2s 252us/step - loss: 0.4270 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 250us/step - loss: 0.4220 - accuracy: 0.8001
Epoch 4/100
8000/8000 [==============================] - 2s 248us/step - loss: 0.4190 - accuracy: 0.8224
Epoch 5/100
8000/8000 [==============================] - 2s 247us/step - loss: 0.4164 - accuracy: 0.8254
Epoch 6/100
8000/8000 [==============================] - 2s 253us/step - loss: 0.4151 - accuracy: 0.8276
Epoch 7/100
8000/8000 [==============================] - 2s 255us/step - loss: 0.4135 - accuracy: 0.8316
Epoch 8/100
8000/8000 [==============================] - 2s 251us/step - loss: 0.4119 - accuracy: 0.8307
Epoch 9/100
8000/8000 [==============================] - 2s 248us/step - loss: 0.4108 - accuracy: 0.8334
Epoch 10/100
8000/8000 [==============================]

In [15]:
# predicting in test set
Y_pred=classifier.predict(X_test)
Y_pred

array([[0.1905415 ],
       [0.33841935],
       [0.15241545],
       ...,
       [0.20395304],
       [0.13962168],
       [0.08520483]], dtype=float32)

In [16]:
#All greater than 0.5 is true less than 0.5 is false
Y_pred=(Y_pred>0.5)

In [17]:
#  creating confusion matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test, Y_pred)
cm

array([[1546,   49],
       [ 261,  144]])

In [18]:
# new prediction
#in horizontal matrix (using double [[]]) - creating 2d array
new_prediction=classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction=(new_prediction>0.5)
new_prediction

array([[False]])

In [19]:
#K-fold cross validation
#Combine keras and scikitlearn - keras wrapper wraps kfold cross validation inside the keras package
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

In [20]:
#use a function for kerasclassifier - a fuction returning and building on ANN architecture
#def defines function - builds ANN classifier
#only use the part of code that builds the ANN not the last part
def build_classifier():
  classifier=Sequential()
  classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=11))
  classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
  classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
  classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return classifier
#this returns a classifer for local and we need to make a global output using wrapper
classifier = KerasClassifier(build_fn =build_classifier, batch_size=10, nb_epoch=100)
#new obejct to contain the accuracy of 10 diffrent cross validation scores
#n_jobs for number of cpu used -1 is for all CPU
accuracies=cross_val_score(estimator = classifier, X = X_train, y=Y_train, cv=10, n_jobs = -1)

In [21]:
accuracies

array([0.78625   , 0.79000002, 0.80000001, 0.78625   , 0.81625003,
       0.81      , 0.78750002, 0.79374999, 0.79874998, 0.79500002])

In [23]:
mean=accuracies.mean()
variance=accuracies.std()


0.009657416205378664